In [1]:
import pandas as pd
%matplotlib inline

import numpy as np
import glob

import datetime
import warnings

from pre_process_sectors_categories import *

%load_ext autoreload
%autoreload 2

# from wb_api_wrapper import *


In [2]:
glob.glob("consumption_and_household_surveys/*")

date_tag = glob.os.path.split(glob.glob("consumption_and_household_surveys/*")[-1])[-1]
hhh_dir = "consumption_and_household_surveys/"+date_tag+"/"
hhh_dir

'consumption_and_household_surveys/2017-10-13/'

# Read household survey with individual job classification

In [3]:
db = pd.read_csv(hhh_dir+"BRA_POF_2008-2009_PERSONA_with_isic.csv")
db.head()

,Unnamed: 0,pais,anio,encuesta,cod_hogar,cod_persona,miembros_hogar,factor_expansion,ing_lab_persona,rama_pri,...,gasto_edre,gasto_otros,cod_uf,num_seq,num_dv,cod_domc,num_ordem_inform,rama_pri_cor,rama_pri_isic3,rama_pri_isic4
0,0,BRA,2008,POF,1.0,1,5,421.778564,0.000000,NaN,...,88.650002,264.746674,11,1,9,1,4.0,NaN,NaN,NaN
1,1,BRA,2008,POF,1.0,2,5,421.778564,5310.521484,53068.0,...,88.650002,264.746674,11,1,9,1,1.0,53068.0,"513, 514, 519, 523, 524","4641,4649,4663,4661,4662,4669,4690,4772,4751,4..."
2,2,BRA,2008,POF,1.0,3,5,421.778564,0.000000,NaN,...,88.650002,264.746674,11,1,9,1,2.0,NaN,NaN,NaN
3,3,BRA,2008,POF,1.0,4,5,421.778564,474.700043,53064.0,...,88.650002,264.746674,11,1,9,1,3.0,53064.0,"515, 523","4651,4652,4653,4659,4772,4751,4771,4742,4753,4..."
4,4,BRA,2008,POF,2.0,1,3,421.778564,8275.000000,18001.0,...,851.560303,804.262512,11,1,9,3,1.0,18001.0,"181, 182","1313,1410,3290,1420,1511"


In [4]:
db=db.sort_index()
db.head()

,Unnamed: 0,pais,anio,encuesta,cod_hogar,cod_persona,miembros_hogar,factor_expansion,ing_lab_persona,rama_pri,...,gasto_edre,gasto_otros,cod_uf,num_seq,num_dv,cod_domc,num_ordem_inform,rama_pri_cor,rama_pri_isic3,rama_pri_isic4
0,0,BRA,2008,POF,1.0,1,5,421.778564,0.000000,NaN,...,88.650002,264.746674,11,1,9,1,4.0,NaN,NaN,NaN
1,1,BRA,2008,POF,1.0,2,5,421.778564,5310.521484,53068.0,...,88.650002,264.746674,11,1,9,1,1.0,53068.0,"513, 514, 519, 523, 524","4641,4649,4663,4661,4662,4669,4690,4772,4751,4..."
2,2,BRA,2008,POF,1.0,3,5,421.778564,0.000000,NaN,...,88.650002,264.746674,11,1,9,1,2.0,NaN,NaN,NaN
3,3,BRA,2008,POF,1.0,4,5,421.778564,474.700043,53064.0,...,88.650002,264.746674,11,1,9,1,3.0,53064.0,"515, 523","4651,4652,4653,4659,4772,4751,4771,4742,4753,4..."
4,4,BRA,2008,POF,2.0,1,3,421.778564,8275.000000,18001.0,...,851.560303,804.262512,11,1,9,3,1.0,18001.0,"181, 182","1313,1410,3290,1420,1511"


# count jobs

In [5]:
db.rama_pri_isic4.dropna().head()

1    4641,4649,4663,4661,4662,4669,4690,4772,4751,4...
3    4651,4652,4653,4659,4772,4751,4771,4742,4753,4...
4                             1313,1410,3290,1420,1511
5                        8610,8620,8690,8710,8720,8730
6                             8421,8890,8422,7120,8423
Name: rama_pri_isic4, dtype: object

In [6]:
ocupation_splitted = pd.DataFrame(split_series_into_lines(db.rama_pri_isic4.dropna()), columns=["isic" ])
ocupation_splitted.head()

,isic
1,4641
1,4649
1,4663
1,4661
1,4662


In [7]:
number_of_possible_isic = ocupation_splitted.groupby(level=[0]).size()
number_of_possible_isic.head()

1    21
3    17
4     5
5     6
6     5
dtype: int64

In [8]:
ocupation_splitted["weight"]= 1/ number_of_possible_isic
ocupation_splitted.head()

,isic,weight
1,4641,0.047619
1,4649,0.047619
1,4663,0.047619
1,4661,0.047619
1,4662,0.047619


In [9]:
# check: should be one
ocupation_splitted.weight.sum(level=0).sample(10)

18676     1.0
10226     1.0
62799     1.0
71034     1.0
124388    1.0
123335    1.0
66884     1.0
43446     1.0
61056     1.0
107612    1.0
Name: weight, dtype: float64

In [10]:
ocupation_splitted["factor_expansion"] = db["factor_expansion"]

In [11]:
jobs_per_sector = ocupation_splitted.set_index("isic")[["weight","factor_expansion"]].prod(axis=1).sum(level=0)

In [12]:
glob.os.makedirs(hhh_dir+"jobscount",exist_ok=True)

In [13]:
jobs_per_sector.index.name="isic4"
jobs_per_sector.name = "jobs_count_BRA"
jobs_per_sector.to_csv(hhh_dir+"jobscount"+"/BRA.csv",header=True)

# Compare with total number of jobs from IADB

In [14]:
jobs_per_sector.sum()*1e-6

88.13422545997561

In [15]:
search_wb("population")

NameError: name 'search_wb' is not defined

In [ ]:
wb_data =pd.DataFrame()
wb_data["population"]= get_wb("SP.POP.TOTL")["2008"]

In [ ]:
sims_data = pd.read_csv("sims_data/RawDataExport20171030.csv", encoding="ansi", skiprows=14,usecols=[0,1,2,10] , squeeze=True,  converters={"valor":lambda s:s.replace(",","")}
                       ).set_index(["extended_country","anio","var_description"]).squeeze()

In [ ]:
sims_data["Brazil"][2008]

In [ ]:
wb_data.loc["Brazil","population"]*sims_data["Brazil"][2008].astype(float)["Employed population (%)"]/100